In [1]:
import pandas as pd

In [2]:
import tensorflow as tf

In [3]:
import os
os.environ['TF_METAL_PLUGGABLE_DEVICE'] = '1'

In [4]:
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [5]:
replay_buf, data_stats = pd.read_pickle('../Research/data/replay_buffer.df'), pd.read_pickle('new_data/data_stats.df')

In [6]:
# def preprocess_booking_good(filename, save_name, itm2idx=None):
#     df = pd.read_csv(filename)
#     df.columns = ['session_id', 'timestamp', 'checkout', 'item_id', 'time_at', 'is_buy']
#     df = df.drop(['checkout', 'time_at'], axis=1)
#     df['timestamp'] = pd.to_datetime(df['timestamp'])
#     # df['valid_session'] = df.session_id.map(df.groupby('session_id')['item_id'].size() > 2)
#     # df = df.loc[df.valid_session].drop('valid_session', axis=1)
#     if itm2idx is None:
#         itm2idx = {k: i for i, k in enumerate(set(df['item_id'].unique()))}
#     idx2itm = {v: k for k, v in itm2idx.items()}
#     df['item_id'] = df['item_id'].map(itm2idx)
#     sorted_df = df.sort_values(by=['session_id', 'timestamp'])
#     #sorted_df.to_csv(save_name, index=None, header=True)
#     return sorted_df, itm2idx, idx2itm

In [7]:
# df_train_booking, itm2idx, idx2itm = preprocess_booking_good('my-time-mdt-buy.csv', save_name='sorted_travels.csv')
# df_test_booking, _, idx2itm  = preprocess_booking_good('test-my-time-mdt-buy.csv', save_name='test_sorted_travels.csv', itm2idx=itm2idx)

In [8]:
data_stats

,state_size,item_num
0,40,39901


In [9]:
from MY.RL_RECSYS import RecQ

In [10]:
arg_dict = {'r_click' : 0.5,
            'r_buy' : 1,
            'r_negative' : 1,
            'hidden_factor' : 64,
            'lr' : 0.005,
            'epoch' : 1,
            'batch_size' : 512,
            'neg' : 30,
            'discount' : 0.8
            }

In [11]:
data_stats['item_num'][0]

39901

In [12]:
%load_ext autoreload
%autoreload 2 

In [13]:
from MY.RL_RECSYS import RecQ, train_recq


QN = RecQ(item_num=data_stats['item_num'][0], state_size=data_stats['state_size'][0], arg_dict=arg_dict)
QN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005))
train_recq(data_stats, replay_buf, arg_dict)

2024-05-12 16:03:39.676571: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-05-12 16:03:39.679644: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-05-12 16:03:39.679700: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-05-12 16:03:39.681318: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-12 16:03:39.684970: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


$$$$ STARTING EPOCH # 0 $$$$


KeyboardInterrupt: 